In [2]:
#Question 1a

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import mean_squared_error

wine = pd.read_csv('wine.txt', sep = '\t')
wine.head()

# Extract predictors and response variable
X = wine.drop(columns=["Quality"])
y = wine["Quality"]

# Initialize LOOCV
loocv = LeaveOneOut()

# Initialize an array to store the test MSE for each fold
test_mse = []

# Perform LOOCV
for train_index, test_index in loocv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Fit a linear regression model on the training data
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Make predictions on the test data
    y_pred = model.predict(X_test)

    # Calculate the test MSE for this fold
    mse = mean_squared_error(y_test, y_pred)
    test_mse.append(mse)
    
# Compute the average test MSE across all folds
average_test_mse_a = np.mean(test_mse)
print(average_test_mse_a)


1.7963428432552002


In [36]:
#Question 1b

from itertools import combinations

# Create a list of all predictor variables
predictors = X.columns

# Initialize variables to keep track of the best model and its performance
best_model = None
best_mse = float('inf')
best_adj_r2 = -float('inf')

# Perform best-subset selection
for k in range(1, len(predictors) + 1):
    for subset in combinations(predictors, k):
        # Create a model using the current subset of predictors
        subset_idx = [predictors.get_loc(p) for p in subset]
        X_subset = X.iloc[:, subset_idx]
        
        # Initialize an array to store the test MSE for this subset
        test_mse_subset = []
        
        # Perform LOOCV for the current subset of predictors
        for train_index, test_index in loocv.split(X_subset):
            X_train, X_test = X_subset.iloc[train_index], X_subset.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            
            # Fit a linear regression model on the training data
            model = LinearRegression()
            model.fit(X_train, y_train)
            
            # Make predictions on the test data
            y_pred = model.predict(X_test)
            
            # Calculate the test MSE for this fold
            mse = mean_squared_error(y_test, y_pred)
            test_mse_subset.append(mse)
        
        # Compute the average test MSE across all folds for this subset
        avg_mse_subset = np.mean(test_mse_subset)
        
        # Calculate the adjusted R^2 for this subset
        n = X_subset.shape[0]
        p = len(subset)
        adj_r2 = 1 - (1 - avg_mse_subset) * ((n - 1) / (n - p - 1))
        
        # Check if this model has a better adjusted R^2 and lower test MSE
        if adj_r2 > best_adj_r2:
            best_model = subset
            best_adj_r2 = adj_r2
            best_mse = avg_mse_subset

# Now, fit the best model with all data
best_model_idx = [predictors.get_loc(p) for p in best_model]
X_best = X.iloc[:, best_model_idx]
model = LinearRegression()
model.fit(X_best, y)

# Compute the test MSE of the best model using LOOCV
test_mse_best_model = []

for train_index, test_index in loocv.split(X_best):
    X_train, X_test = X_best.iloc[train_index], X_best.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    test_mse_best_model.append(mse)

# Compute the average test MSE for the best model
average_test_mse_best_subset = np.mean(test_mse_best_model)

# Print the best model and its test MSE
print("Best Subset Model (Predictors):", best_model)
print("Test MSE of the Best Model:", average_test_mse_best_subset)


Best Subset Model (Predictors): ('Clarity', 'Oakiness')
Test MSE of the Best Model: 4.706194265856787


In [37]:
#Question 1c

# Create a list of all predictor variables
predictors = X.columns

# Initialize an empty list to store the selected predictors
selected_predictors = []

# Initialize variables to keep track of the best model and its performance
best_model = None
best_mse = float('inf')
best_adj_r2 = -float('inf')

# Perform forward stepwise selection
while len(selected_predictors) < len(predictors):
    remaining_predictors = [p for p in predictors if p not in selected_predictors]
    
    # Initialize variables to keep track of the best predictor to add and its performance
    best_predictor = None
    best_mse_with_predictor = float('inf')
    best_adj_r2_with_predictor = -float('inf')
    
    for predictor in remaining_predictors:
        # Create a model using the current set of selected predictors and the predictor to be added
        predictors_to_add = selected_predictors + [predictor]
        X_subset = X[predictors_to_add]
        
        # Initialize an array to store the test MSE for this subset
        test_mse_subset = []
        
        # Perform LOOCV for the current subset of predictors
        for train_index, test_index in loocv.split(X_subset):
            X_train, X_test = X_subset.iloc[train_index], X_subset.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            
            # Fit a linear regression model on the training data
            model = LinearRegression()
            model.fit(X_train, y_train)
            
            # Make predictions on the test data
            y_pred = model.predict(X_test)
            
            # Calculate the test MSE for this fold
            mse = mean_squared_error(y_test, y_pred)
            test_mse_subset.append(mse)
        
        # Compute the average test MSE across all folds for this subset
        avg_mse_subset = np.mean(test_mse_subset)
        
        # Calculate the adjusted R^2 for this subset
        n = X_subset.shape[0]
        p = len(predictors_to_add)
        adj_r2 = 1 - (1 - avg_mse_subset) * ((n - 1) / (n - p - 1))
        
        # Check if this predictor addition has a better adjusted R^2 and lower test MSE
        if adj_r2 > best_adj_r2_with_predictor:
            best_predictor = predictor
            best_adj_r2_with_predictor = adj_r2
            best_mse_with_predictor = avg_mse_subset
    
    # Add the best predictor to the selected predictors
    selected_predictors.append(best_predictor)
    
    # Update the best model and its performance
    if best_mse_with_predictor < best_mse:
        best_model = selected_predictors.copy()
        best_mse = best_mse_with_predictor
        best_adj_r2 = best_adj_r2_with_predictor

# Now, fit the best model with all data
X_best = X[best_model]
model = LinearRegression()
model.fit(X_best, y)

# Compute the test MSE of the best model using LOOCV
test_mse_best_model = []

for train_index, test_index in loocv.split(X_best):
    X_train, X_test = X_best.iloc[train_index], X_best.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    test_mse_best_model.append(mse)

# Compute the average test MSE for the best model
average_test_mse_forward = np.mean(test_mse_best_model)

# Print the best model and its test MSE
print("Best Forward Stepwise Model (Predictors):", best_model)
print("Test MSE of the Best Model:", average_test_mse_forward)


Best Forward Stepwise Model (Predictors): ['Clarity', 'Oakiness', 'Region', 'Body', 'Aroma', 'Flavor']
Test MSE of the Best Model: 1.796342843255199


In [46]:
# Question 1d

import numpy as np
import pandas as pd
from itertools import combinations
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import mean_squared_error

# Extract predictors and response variable
X = wine.drop(columns=["Quality"])
y = wine["Quality"]

# Initialize LOOCV
loocv = LeaveOneOut()

# Create a list of all predictor variables
predictors = X.columns

# Initialize the list of selected predictors with all predictors
selected_predictors = predictors.copy()

# Initialize variables to keep track of the best model and its performance
best_model = selected_predictors
best_mse = float('inf')

# Perform backward stepwise selection
while len(selected_predictors) > 0:
    # Initialize variables to keep track of the best predictor to remove and its performance
    worst_predictor = None
    worst_mse_without_predictor = float('inf')
    
    for predictor in selected_predictors:
        # Create a model using the current set of selected predictors without the predictor to be removed
        predictors_to_remove = selected_predictors.copy()
        predictors_to_remove.drop(predictor)
        X_subset = X[predictors_to_remove]
        
        # Initialize an array to store the test MSE for this subset
        test_mse_subset = []
        
        # Perform LOOCV for the current subset of predictors
        for train_index, test_index in loocv.split(X_subset):
            X_train, X_test = X_subset.iloc[train_index], X_subset.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            
            # Fit a linear regression model on the training data
            model = LinearRegression()
            model.fit(X_train, y_train)
            
            # Make predictions on the test data
            y_pred = model.predict(X_test)
            mse = mean_squared_error(y_test, y_pred)
            test_mse_subset.append(mse)
        
        # Compute the average test MSE across all folds for this subset
        avg_mse_subset = np.mean(test_mse_subset)
        
        # Calculate the adjusted R^2 for this subset
        n = X_subset.shape[0]
        p = len(predictors_to_remove)
        adj_r2 = 1 - (1 - avg_mse_subset) * ((n - 1) / (n - p - 1))
        
        # Check if removing this predictor has a better adjusted R^2 and lower test MSE
        if avg_mse_subset < worst_mse_without_predictor:
            worst_predictor = predictor
            worst_mse_without_predictor = avg_mse_subset
    
    # Remove the worst predictor from the selected predictors
    selected_predictors.drop(worst_predictor)
    
    # Update the best model and its performance
    if worst_mse_without_predictor < best_mse:
        best_model = selected_predictors.copy()
        best_mse = worst_mse_without_predictor

# Now, fit the best model with all data
X_best = X[best_model]
model = LinearRegression()
model.fit(X_best, y)

# Compute the test MSE of the best model using LOOCV
test_mse_best_model = []

for train_index, test_index in loocv.split(X_best):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    test_mse_best_model.append(mse)

# Compute the average test MSE for the best model
average_test_mse_best_model = np.mean(test_mse_best_model)

# Print the best model and its test MSE
print("Best Backward Stepwise Model (Predictors):", best_model)
print("Test MSE of the Best Model:", average_test_mse_best_model)

AttributeError: 'LinearRegression' object has no attribute 'coef_'

In [20]:
# Question 1e

from sklearn.linear_model import RidgeCV

# Initialize LOOCV
loocv = LeaveOneOut()

# Create a RidgeCV model with a range of alpha values
alphas = [0.001, 0.01, 0.1, 1, 10, 100]  # Adjust the range as needed
ridge = RidgeCV(alphas=alphas, store_cv_values=True)

# Initialize variables to store results
best_alpha = None
best_test_mse = float('inf')

# Perform LOOCV to find the best alpha
for train_index, test_index in loocv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Fit the RidgeCV model on the training data
    ridge.fit(X_train, y_train)
    
    # Get the optimal alpha value selected by RidgeCV
    alpha = ridge.alpha_
    
    # Evaluate the model on the test data
    y_pred = ridge.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    
    # Update the best alpha and best test MSE if necessary
    if mse < best_test_mse:
        best_alpha = alpha
        best_test_mse = mse

# Fit the Ridge model with the best alpha on the entire dataset
ridge.fit(X, y)

# Compute the test MSE of the model
test_mse_ridge = []

for train_index, test_index in loocv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    ridge.fit(X_train, y_train)
    y_pred = ridge.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    test_mse_ridge.append(mse)

# Compute the average test MSE for the Ridge model
average_test_mse_ridge = np.mean(test_mse_ridge)

# Print the best alpha and the test MSE of the Ridge model
print("Best Alpha for Ridge Regression:", best_alpha)
print("Test MSE of the Ridge Model:", average_test_mse_ridge)



Best Alpha for Ridge Regression: 10.0
Test MSE of the Ridge Model: 1.7313156163921775


In [23]:
# Question 1f

from sklearn.linear_model import LassoCV

# Initialize LOOCV
loocv = LeaveOneOut()

# Create a LassoCV model with a range of alpha values
alphas = [0.001, 0.01, 0.1, 1, 10, 100]  # Adjust the range as needed
lasso = LassoCV(alphas=alphas)

# Initialize variables to store results
best_alpha = None
best_test_mse = float('inf')

# Perform LOOCV to find the best alpha
for train_index, test_index in loocv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Fit the LassoCV model on the training data
    lasso.fit(X_train, y_train)
    
    # Get the optimal alpha value selected by LassoCV
    alpha = lasso.alpha_
    
    # Evaluate the model on the test data
    y_pred = lasso.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    
    # Update the best alpha and best test MSE if necessary
    if mse < best_test_mse:
        best_alpha = alpha
        best_test_mse = mse

# Fit the Lasso model with the best alpha on the entire dataset
# lasso = LassoCV(alpha=best_alpha)
lasso.fit(X, y)

# Compute the test MSE of the model
test_mse_lasso = []

for train_index, test_index in loocv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    lasso.fit(X_train, y_train)
    y_pred = lasso.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    test_mse_lasso.append(mse)

# Compute the average test MSE for the Lasso model
average_test_mse_lasso = np.mean(test_mse_lasso)

# Print the best alpha and the test MSE of the Lasso model
print("Best Alpha for Lasso Regression:", best_alpha)
print("Test MSE of the Lasso Model:", average_test_mse_lasso)


Best Alpha for Lasso Regression: 0.1
Test MSE of the Lasso Model: 1.5866543012142005


In [38]:
# Question 1g
import pandas as pd

# Create a DataFrame to store the results
results = pd.DataFrame(columns=['Model', 'Test MSE'])

# Add results for each model
results = results.append({'Model': 'Linear Regression (All Predictors)', 
                          'Test MSE': average_test_mse_a}, ignore_index=True)

results = results.append({'Model': 'Best Subset Selection',
                          'Test MSE': average_test_mse_best_subset}, ignore_index=True)

results = results.append({'Model': 'Forward Stepwise Selection', 
                          'Test MSE': average_test_mse_forward}, ignore_index=True)

# results = results.append({'Model': 'Backward Stepwise Selection', 
#                           'Test MSE': average_test_mse_backward_stepwise}, ignore_index=True)

results = results.append({'Model': 'Ridge Regression (Optimal Alpha)', 
                          'Test MSE': average_test_mse_ridge}, ignore_index=True)

results = results.append({'Model': 'Lasso Regression (Optimal Alpha)',
                          'Test MSE': average_test_mse_lasso}, ignore_index=True)

# Print the table of results
print(results)

# To find the best model(s), you can sort the DataFrame by Test MSE in ascending order
best_models = results.sort_values(by='Test MSE')

# Print the best model(s)
print("Best Model(s):")
print(best_models.head())


                                Model  Test MSE
0  Linear Regression (All Predictors)  1.796343
1               Best Subset Selection  4.706194
2          Forward Stepwise Selection  1.796343
3    Ridge Regression (Optimal Alpha)  1.731316
4    Lasso Regression (Optimal Alpha)  1.586654
Best Model(s):
                                Model  Test MSE
4    Lasso Regression (Optimal Alpha)  1.586654
3    Ridge Regression (Optimal Alpha)  1.731316
2          Forward Stepwise Selection  1.796343
0  Linear Regression (All Predictors)  1.796343
1               Best Subset Selection  4.706194


C:\Users\annbi\AppData\Local\Temp\ipykernel_11088\3214059107.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'Model': 'Linear Regression (All Predictors)',
C:\Users\annbi\AppData\Local\Temp\ipykernel_11088\3214059107.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'Model': 'Best Subset Selection',
C:\Users\annbi\AppData\Local\Temp\ipykernel_11088\3214059107.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'Model': 'Forward Stepwise Selection',
C:\Users\annbi\AppData\Local\Temp\ipykernel_11088\3214059107.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat in